In [1]:
from ratdata import data_manager as dm, process, ingest
import pathlib

In [2]:
dm.db_connect('rat_data.db')

In [3]:
recording_files = dm.RecordingFile.select()

In [ ]:
for i, f in enumerate(recording_files):
    fname = pathlib.Path(f.dirname) / f.filename
    recording = ingest.read_mce_matlab_file(str(fname))
    recording_beta_power = process.compute_power_in_frequency_band(recording.electrode_data.mean(0), 10, 30, int(1/recording.dt))
    recording_total_power = process.compute_power_in_frequency_band(recording.electrode_data.mean(0), 1, 100, int(1/recording.dt))
    rp_id = dm.RecordingPower.replace(recording=f, beta_power=recording_beta_power, total_power=recording_total_power).execute()
    print('[%d/%d] %s: %d' % (i + 1, len(recording_files), f.filename, rp_id))
    